In [1]:
import numpy as np
import ot
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# https://www.slideshare.net/joisino/ss-251328369
# p35

In [3]:
ws = np.array([0.2, 0.5, 0.2, 0.1])
wt = np.array([0.3, 0.3, 0.4, 0.0])

# https://stackoverflow.com/questions/58584413/black-formatter-ignore-specific-multi-line-code
# fmt: off
C = np.array([[0, 2, 2, 2],
              [2, 0, 1, 2],
              [2, 1, 0, 2],
              [2, 2, 2, 0]])
# fmt: on

In [4]:
# 汎用ソルバ・厳密ソルバによる解法
# 線形計画問題として定式化→シンプレックス法とかで解く
P_ = ot.emd(ws, wt, C)
cost = ot.emd2(ws, wt, C)  # (P * C).sum()
print(P_)
print(cost)

[[0.2 0.  0.  0. ]
 [0.  0.3 0.2 0. ]
 [0.  0.  0.2 0. ]
 [0.1 0.  0.  0. ]]
0.39999999999999997


In [5]:
"""
sinkhornでの解法
- 高速
- シンプル
- ただし、厳密な最適輸送の解は求まらない
"""
eps = 0.1  # 大きいと高速、小さいと厳密に
K = np.exp(-C / eps)  # Gibbs kernel
"""
u = exp(f/eps)
v = exp(g/eps)
"""
u = np.ones(4)
for i in range(100):
    v = wt / (K.T @ u)
    u = ws / (K @ v)
# f = eps * np.log(u + 1e-9)  # シンクホーン変数から元の双対変数へ
# g = eps * np.log(v + 1e-9)
"""
P = exp((f + g - C)/eps)
f, gが完璧に収束しない限り、Pは厳密には実行可能とは限らない
"""
P = u.reshape(4, 1) * K * v.reshape(1, 4)  # 主問題の解
np.set_printoptions(formatter={"float": "{:.2f}".format})
print(P)
print((P * C).sum())

[[0.20 0.00 0.00 0.00]
 [0.00 0.30 0.20 0.00]
 [0.00 0.00 0.20 0.00]
 [0.10 0.00 0.00 0.00]]
0.3993214951621453


In [10]:
"""
https://www.slideshare.net/joisino/ss-249394573
p218
シンクホーンは微分可能（特に、計算が行列演算のみのため、自動微分ライブラリが使える）
微分の使いどころの例：配置問題
倉庫: nコ
工場: mコ
j番目の工場の位置は(x_j, y_j)で、b_jグラムの小麦粉を要求
i番目の倉庫は小麦粉をa_iグラム保存可能
（倉庫から工場への総輸送コストを最小化するには、輸送行列をどのように設定し、）
各倉庫をどこに配置すればよいか？
→2つの問題を解く？

解き方
1.倉庫の位置変数をランダム初期化
2.C_i_j = 倉庫iと工場jの距離として計算
3.sinkhornに(a, b, C)を入力して総輸送コストを評価
4.倉庫の位置変数の勾配を求めて倉庫の位置を改善→2へ
"""

torch.manual_seed(0)

x = torch.rand(20, 2)
y = torch.rand(20, 2) + torch.FloatTensor([0, 2])
z = torch.rand(20, 2) + torch.FloatTensor([1, 1])
mu = torch.cat([x, y, z])
# step 1
# nu = nn.parameter.Parameter(torch.rand(12, 2) * 2)  # 位置変数のつもり
nu = torch.rand(12, 2) * 2
nu.requires_grad = True
n, m = len(mu), len(nu)
a = torch.ones(n) / n
b = torch.ones(m) / m

eps = 0.1
optimizer = optim.SGD([nu], lr=1.0)
for it in range(100):
    # step 2
    C = torch.linalg.norm(mu.reshape(n, 1, 2) - nu.reshape(1, m, 2), axis=2)
    K = torch.exp(-C / eps)
    u = torch.ones(n)
    for i in range(100):
        v = b / (K.T @ u)
        u = a / (K @ v)

    # f = eps * torch.log(u + 1e-9)
    # g = eps * torch.log(v + 1e-9)

    # step 3
    P = u.reshape(n, 1) * K * v.reshape(1, m)
    loss = (P * C).sum()
    # step 4
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(P)
print(loss)
P_ = ot.emd(a, b, C)
cost = ot.emd2(a, b, C)  # (P * C).sum()
print(P_)
print(cost)

torch.Size([60, 2])
torch.Size([12, 2])
tensor([[6.9870e-10, 6.3203e-09, 1.4194e-05, 7.5215e-03, 9.4084e-04, 2.8546e-08,
         8.1635e-03, 1.5551e-07, 2.6114e-05, 1.1184e-11, 6.6599e-09, 2.5903e-07],
        [5.2709e-13, 1.3678e-11, 1.6395e-02, 6.1183e-06, 2.5540e-04, 2.3140e-11,
         1.0560e-05, 6.8704e-11, 1.4661e-08, 8.1435e-15, 4.1339e-12, 3.0694e-10],
        [2.1971e-10, 3.6507e-09, 1.8135e-04, 1.3618e-03, 1.0318e-02, 9.5330e-09,
         4.8013e-03, 2.6495e-08, 4.0934e-06, 3.4121e-12, 1.0438e-09, 4.3651e-08],
        [2.1769e-09, 1.9584e-08, 4.2549e-06, 4.8403e-03, 2.7931e-04, 8.9787e-08,
         1.1499e-02, 3.5152e-07, 4.2833e-05, 3.4628e-11, 1.1338e-08, 3.1152e-07],
        [2.9601e-10, 3.1187e-09, 9.3654e-05, 5.4865e-03, 5.8626e-03, 1.2256e-08,
         5.2098e-03, 7.0382e-08, 1.3869e-05, 4.7109e-12, 3.6265e-09, 1.8229e-07],
        [2.7667e-11, 3.9863e-10, 1.1544e-03, 5.7255e-04, 1.4335e-02, 1.1773e-09,
         6.0309e-04, 6.1519e-09, 1.3729e-06, 4.3433e-13, 3.8445e

/Users/hiroyoshinagao/Desktop/ot_cv/.venv/lib/python3.9/site-packages/ot/lp/__init__.py:547: UserWarning: Problem infeasible. Check that a and b are in the simplex
  check_result(result_code)
